# 17 • Consulta de tweets

En este notebook se revisará cómo consultar tweets usando Python.

## Contenido
1. Intro  
   1.1. Twitter API   
   1.2. Librería `GetOldTweets3`  
   1.3. Librería `snscrape`  
2. Referencias  

In [1]:
# Import basic libraries
import pandas as pd
import numpy as np
import altair as alt

## 1. Intro
Existen distintas librerías para consultar tweets y cada una de ellas tiene sus ventajas y desventajas. La manera oficial es hacerlo a través de la aplicación oficial de Twitter obteniendo acceso como desarrollador (`Twitter API`) para la cual se requiere solicitar una cuenta. Existen algunas alternativas que en lo personal he utilizado, aunque para usarlas hay que familiarizarse con ellas, como `GetOldTweets3` y `snscrape` pues no están tan bien documentadas.

Es importante mencionar que las consultas de twitter usualmente arrojan información en formato `json`, los cuales siguen una estructura parecida a los diccionarios de Python y, en caso de guardar estos archivos, se recomienda guardarlos en formato `.json` en el lugar de convertirlos a `.xls`, `.csv`, `.txt` o `npy` para no perder información.

### 1.1 Twitter API
Esta cuenta no tiene costo, aunque sí existe limitación en número de tweets consultados por día. El primer paso es [solicitar una cuenta de desarrollador](https://developer.twitter.com/en/portal/dashboard). Para quien esté interesado en obtener una cuenta les comparto un par de links que les podrían ser útiles

- [How to get TWEETS by Python, Twitter API 2022](https://www.youtube.com/watch?v=Lu1nskBkPJU) por AI Spectrum.
- [Twitter API Data Collection](https://www.youtube.com/watch?v=Jl-_dDqSaUQ&t=59s) por Stevesie Data, ver del minuto 0:59 al 1:44.
- [Developer account support](https://developer.twitter.com/en/support/twitter-api/developer-account) de Twitter.

Una vez que hayan obtenido su solicitud para abrir cuenta de desarrollador haya sido aceptada, podrán encontrar su llave de autenticación en este [link](https://developer.twitter.com/en/portal/projects-and-apps): `API key`, `API secret key`, `Access token` y `Access token secret`, y con ellas podrán tener acceso con distintos programas, entre ellos con Python.

La API de Twitter algunas limitaciones para consultar tweets descritas en este [link](https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets).

⚠️ __Desventajas__, el obtener una cuenta de desarrollador en Twitter puede tardar tiempo y además de tener restricciones como un tiempo máximo de días previos de consulta (e.g. no se puede consultar tweets de principios de año), además de un número de existir un máximo número de tweets.

### 1.2 Librería `GetOldTweets3`
Esta es una librería está relativamente bien documentada donde se pueden conusltar tweets relativamente antiguos. Para más información pueden consultar la [descripción del paquete](https://pypi.org/project/GetOldTweets3/) con las indicaciones para su instalación y la [página del paquete en GitHub](https://github.com/Mottl/GetOldTweets3).

Es importante mencionar que actualmente este paquete no funciona.

In [2]:
## librerías
# !pip install GetOldTweets3 #<- installa la librería
import GetOldTweets3 as got

In [3]:
# Partámetros para consulta
tweetCriteria = got.manager.TweetCriteria().setUsername("JustinTrudeau")\
                                           .setTopTweets(True)\
                                           .setMaxTweets(10)

In [4]:
# esto se almacena en un objeto
tweetCriteria

In [5]:
# # Al realizar la consulta arroja error
# tweet = got.manager.TweetManager.getTweets(tweetCriteria)[0]
# print(tweet.text)

⚠️ __Desventajas__, actualmente este paquete no funciona de forma correcta. En la siguiente liga se comparte el error, el cual ha sido persistente desde septiembre 2020 https://github.com/Mottl/GetOldTweets3/issues/98

### 1.3 Librería `snscrape`
Esta librería es la mejor alternativa que he utilizado para bajar tweets, aunque aún falta mejorar la documentación. Además, esta librería tiene módulos específicos que sirven para consultar información de otras redes sociales entre las que se encuentran Facebook, Instagram y Telegram.

#### Ejemplo: Obtener tweets relacionados con el **COVID** realizados desde las cuentas de los dirigentes de México, USA y Canadá

In [6]:
# librerías
import snscrape.modules.twitter as sntwitter

In [15]:
# parametros
maxTweets = 2_000
date_initial = "2020-01-01"
date_final = "2023-02-28"

# twitter's accounts
cuentas=['JustinTrudeau', 'JoeBiden', 'lopezobrador_']

In [16]:
%%time
# Parámetros

# Creating list to append tweet data to
tweets_list = []
i=0

# Using TwitterSearchScraper to scrape data and append tweets to list
print("\n--- START: TwitterSearchScraper por cuenta de Twitter ---\n")
for a in cuentas:
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper('"covid" since:'+date_initial+' until:'+date_final+' from:'+a).get_items()):
        if i>maxTweets-1:
            break
        tweets_list.append([tweet.user.username, tweet.date, tweet.id, tweet.content, tweet.url, tweet.lang,
                    tweet.hashtags, tweet.likeCount, tweet.replyCount, tweet.retweetCount, tweet.quoteCount])
        # Otra info que puede sernos útil: tweet.media,  tweet.url
    print("Se identificaron {0} tweets de la cuenta {1} con los parametros dados.".format(i, a))
print("\n--- END: TwitterSearchScraper ---\n\nTIMING")


--- START: TwitterSearchScraper por cuenta de Twitter ---



<timed exec>:13: DeprecatedFeatureWarning: content is deprecated, use rawContent instead


Se identificaron 1239 tweets de la cuenta JustinTrudeau con los parametros dados.
Se identificaron 255 tweets de la cuenta JoeBiden con los parametros dados.
Se identificaron 2 tweets de la cuenta lopezobrador_ con los parametros dados.

--- END: TwitterSearchScraper ---

TIMING
CPU times: user 923 ms, sys: 70.2 ms, total: 994 ms
Wall time: 55.7 s


In [17]:
# Pandas dataframe con tweets de los tres presidentes relacionados con el Covid-19
column_names = ("username","date","id","content","url","language","hashtags",
                "likes_count","reply_count","retweet_count","quote_count")
df = pd.DataFrame(tweets_list, columns=column_names)
df

,username,date,id,content,url,language,hashtags,likes_count,reply_count,retweet_count,quote_count
0,JustinTrudeau,2022-12-26 21:00:54+00:00,1607481604850450433,The best way to stay healthy – and to stay in ...,https://twitter.com/JustinTrudeau/status/16074...,en,None,4876,20611,750,1338
1,JustinTrudeau,2022-12-26 20:51:31+00:00,1607479240114438150,La meilleure façon de rester en bonne santé – ...,https://twitter.com/JustinTrudeau/status/16074...,fr,None,436,600,57,23
2,JustinTrudeau,2022-12-15 01:07:07+00:00,1603194909841817601,Let’s keep doing everything we can to stay saf...,https://twitter.com/JustinTrudeau/status/16031...,en,None,1660,4355,248,196
3,JustinTrudeau,2022-12-15 01:07:02+00:00,1603194891927814144,Continuons de faire tout ce que nous pouvons p...,https://twitter.com/JustinTrudeau/status/16031...,fr,None,219,233,33,7
4,JustinTrudeau,2022-11-09 23:47:41+00:00,1590491348074782720,"It’s safe, it’s easy, and it’s free. Find out ...",https://twitter.com/JustinTrudeau/status/15904...,en,None,2136,4804,394,385
...,...,...,...,...,...,...,...,...,...,...,...
1494,JoeBiden,2020-03-15 14:46:19+00:00,1239201266410164224,If you're exhibiting symptoms of COVID-19 — or...,https://twitter.com/JoeBiden/status/1239201266...,en,None,5456,360,1290,123
1495,JoeBiden,2020-03-15 14:43:52+00:00,1239200647616004096,The right to vote is the most sacred American ...,https://twitter.com/JoeBiden/status/1239200647...,en,None,16995,3539,3684,2649
1496,lopezobrador_,2021-01-25 00:30:45+00:00,1353500519344578560,Lamento informarles que estoy contagiado de CO...,https://twitter.com/lopezobrador_/status/13535...,es,None,114772,54853,28578,24897
1497,lopezobrador_,2020-12-27 02:24:20+00:00,1343019855272931336,Nos reunimos con la jefa de Gobierno e integra...,https://twitter.com/lopezobrador_/status/13430...,es,None,13114,1768,3178,323


In [18]:
# ejemplo de tweet
df['content'][0]

'The best way to stay healthy – and to stay in the game – is to keep your COVID-19 vaccinations up to date. If you’re due for one of your shots, click here and find where and when you can get it: https://t.co/bIkQIX6btP https://t.co/E7H1NC13js'

In [20]:
# Número de comentarios por presidente/primer ministro
print("\nTweets de presidente o primer ministro relacionados con el Covid-19, de enero 2020 a febrero 2023:\n")
pd.DataFrame(df['username'].value_counts()).reset_index().rename(columns={'index':'President', 'username':"tweets"})


Tweets de presidente o primer ministro relacionados con el Covid-19, de enero 2020 a febrero 2023:



,President,tweets
0,JustinTrudeau,1240
1,JoeBiden,256
2,lopezobrador_,3


#### Ejemplo: Obtener tweets con hashtag #INENoSeToca

In [21]:
%%time

# Parámetros
tweets_list_ine = []
maxTweets_ine = 10_000
date_initial = "2023-01-01"

# Get tweets
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('#INENoSeToca').get_items()): # se puede añadir esto --> since:'+date_initial
        if i>maxTweets_ine-1:
            break
        tweets_list_ine.append([tweet.user.username, tweet.date, tweet.id, tweet.content, tweet.url, tweet.lang,
                    tweet.hashtags, tweet.likeCount, tweet.replyCount, tweet.retweetCount, tweet.quoteCount])
        # Otra info que puede sernos útil: tweet.media,  tweet.url

<timed exec>:10: DeprecatedFeatureWarning: content is deprecated, use rawContent instead


CPU times: user 6.18 s, sys: 462 ms, total: 6.64 s
Wall time: 9min 28s


In [22]:
# Pandas dataframe con tweets que mencionen el hashtag #INENoSeToca
df_ine = pd.DataFrame(tweets_list_ine, columns=column_names)
df_ine['date'] = df_ine['date'].dt.strftime('%Y-%m-%d')
df_ine

,username,date,id,content,url,language,hashtags,likes_count,reply_count,retweet_count,quote_count
0,Cardinalboy3,2023-03-22,1638646056782290944,@AztecDuncan @lorenzocordovav @INEMexico @lila...,https://twitter.com/Cardinalboy3/status/163864...,qme,[inenosetoca],0,0,0,0
1,ALFREDOMARTNEZ8,2023-03-22,1638620226919997470,O que crean defender causas como el #INENoSeTo...,https://twitter.com/ALFREDOMARTNEZ8/status/163...,es,[INENoSeToca],0,0,0,0
2,InnovationChief,2023-03-22,1638593032432885773,Esta tipa de Berta Alcalde nos quiere retroced...,https://twitter.com/InnovationChief/status/163...,es,"[INE, inenosetoca]",0,0,0,0
3,pedroskiro2,2023-03-22,1638578239013638147,"@HernanGomezB Tendré que adquirir ese libro, q...",https://twitter.com/pedroskiro2/status/1638578...,es,[INEnoSeToca],0,0,0,0
4,punetasboy,2023-03-22,1638515236461215746,@PartidoMorenaMx Ya se le dijo que el #INENoSe...,https://twitter.com/punetasboy/status/16385152...,es,"[INENoSeToca, NAIM, Populismo, ignorancia]",0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,renekmp,2022-11-13,1591857296707141632,#INENoSeToca https://t.co/lPVUfXHLpf,https://twitter.com/renekmp/status/15918572967...,qme,[INENoSeToca],1,0,0,0
9996,aleed99,2022-11-13,1591857284057030656,#INENoSeToca,https://twitter.com/aleed99/status/15918572840...,qht,[INENoSeToca],0,0,0,0
9997,Martires2000,2022-11-13,1591857282169835521,Yo defiendo la democracia desde Morelia Michoa...,https://twitter.com/Martires2000/status/159185...,es,"[ElINESeDefiende, ElINEesMx, INENoSeToca]",0,0,0,0
9998,Alb3rtoCano,2022-11-13,1591857275022774273,#INENoSeToca 👊🏻,https://twitter.com/Alb3rtoCano/status/1591857...,und,[INENoSeToca],0,0,0,0


In [23]:
# ejemplo de tweet
df_ine['content'][1]

'O que crean defender causas como el #INENoSeToca: consigna emitida desde la presidencia del consejo, y se vio que la mayoría de la "marea rosa" no tenía idea de qué trataba las reformas del PlanB. Nunca van aceptar que es el odio el que los mueve'

In [24]:
source = pd.DataFrame(df_ine['date'].value_counts()).reset_index().rename(columns={'index':'fecha', 'date':"tweets"})
source.head()

,fecha,tweets
0,2022-11-13,2894
1,2022-11-14,2729
2,2022-11-15,632
3,2022-12-07,499
4,2022-11-16,386


In [25]:
alt.Chart(source[source['fecha'] >= "2023-01-01"]).mark_area(color="pink").encode(
    alt.X("fecha:T", title="2022"),
    alt.Y("tweets:Q", title="Número de Tweets", scale=alt.Scale(type="log")),
    tooltip=["fecha:T","tweets"]
).properties(
    title="Volumen de tweets con hashtag #INENoSeToca durante 2023"
)

alt.Chart(...)

# Referencias
- **API de Twitter**
    - [Solicitud de cuenta de desarrollador](https://developer.twitter.com/en/portal/dashboard)
    - [Twitter API Data Collection](https://www.youtube.com/watch?v=Jl-_dDqSaUQ&t=59s) por Stevesie Data (minuto 0:59 al 1:44)
    - [Ejemplo de uso](https://github.com/vcuspinera/Canada_response_covid/blob/master/src/twitter-search_v1_TwitterAPI.ipynb)  

<br>

- **librería `GetOldTweets`**
    - [Descripción del paquete](https://pypi.org/project/GetOldTweets3/) 
    - [Página del paquete en GitHub](https://github.com/Mottl/GetOldTweets3)
    - [Error del paquete](https://github.com/Mottl/GetOldTweets3/issues/98)
    - [Ejemplo de uso](https://github.com/vcuspinera/Canada_response_covid/blob/master/src/twitter-search_v2_GetOldTweets3.ipynb)  

<br>

- **librería `snscrape`**
    - [Descripción del paquete](https://pypi.org/project/snscrape/) 
    - [Página del paquete en GitHub](https://github.com/JustAnotherArchivist/snscrape)
    - [Ejemplo de uso](https://github.com/vcuspinera/Canada_response_covid/blob/master/src/twitter-search_v3_snscrape.ipynb)  

<br>

- **Artículos**
    - [How to scrape millions of tweets using snscrape](https://medium.com/dataseries/how-to-scrape-millions-of-tweets-using-snscrape-195ee3594721) por Rashi Desai.
    - [How to Scrape Tweets With snscrape](https://betterprogramming.pub/how-to-scrape-tweets-with-snscrape-90124ed006af) por Martin Beck.